# Buy and Hold Strategy

This notebook demonstrates a simple buy-and-hold strategy with comprehensive analysis:
- Configure any stock or ETF symbol (SPY, AAPL, MSFT, etc.)
- Buy on the first trading day and hold until the end
- Track performance over time
- Generate detailed pyfolio tearsheet with risk metrics

## Setup

In [ ]:
# Register Sharadar bundle (required for Jupyter notebooks)
from zipline.data.bundles import register
from zipline.data.bundles.sharadar_bundle import sharadar_bundle

register('sharadar', sharadar_bundle(tickers=None, incremental=True, include_funds=True))
print("✓ Sharadar bundle registered")

In [ ]:
import logging
import pandas as pd
import matplotlib.pyplot as plt

from zipline import run_algorithm
from zipline.api import (
    order,
    order_target_percent,
    symbol,
    record,
    schedule_function,
    date_rules,
    time_rules,
)
from zipline.utils.progress import enable_progress_logging
from zipline.utils.flightlog_client import enable_flightlog

# Enable logging
logging.basicConfig(level=logging.INFO, force=True)

# Connect to FlightLog (optional - remove if not using)
# enable_flightlog(host='flightlog', port=9020)

# Enable progress tracking
enable_progress_logging(algo_name='Buy-and-Hold', update_interval=10)

## Configuration

Change the symbol below to backtest different stocks or ETFs.

In [ ]:
# CONFIGURATION: Change the symbol to backtest different stocks
# Examples: 'SPY', 'AAPL', 'MSFT', 'QQQ', 'NVDA', 'TSLA'
SYMBOL = 'SPY'

print(f"✓ Symbol configured: {SYMBOL}")

## Strategy Implementation

Buy the selected stock on the first day and hold until the end.

In [ ]:
def initialize(context):
    """
    Called once at the start of the algorithm.
    """
    # Set the stock to trade (use the configured SYMBOL)
    context.asset = symbol(SYMBOL)
    
    # Track if we've made our initial purchase
    context.has_bought = False
    
    logging.info(f"Strategy initialized. Trading: {context.asset.symbol}")


def handle_data(context, data):
    """
    Called every day (or bar).
    """
    # Buy on the first day (only if tradeable)
    if not context.has_bought and data.can_trade(context.asset):
        # Get current price
        price = data.current(context.asset, 'price')
        
        # Invest 100% of portfolio in the asset
        order_target_percent(context.asset, 1.0)
        
        context.has_bought = True
        logging.info(f"Bought {context.asset.symbol} at ${price:.2f}")
    
    # Record daily values (only if we have data)
    if data.can_trade(context.asset):
        record(
            price=data.current(context.asset, 'price'),
            portfolio_value=context.portfolio.portfolio_value
        )
    else:
        record(portfolio_value=context.portfolio.portfolio_value)


def analyze(context, perf):
    """
    Called after the backtest completes.
    """
    logging.info("\n" + "="*60)
    logging.info("BACKTEST RESULTS")
    logging.info("="*60)
    
    # Calculate returns
    start_value = perf['portfolio_value'].iloc[0]
    end_value = perf['portfolio_value'].iloc[-1]
    total_return = (end_value - start_value) / start_value * 100
    
    logging.info(f"Start date: {perf.index[0].date()}")
    logging.info(f"End date: {perf.index[-1].date()}")
    logging.info(f"")
    logging.info(f"Starting portfolio value: ${start_value:,.2f}")
    logging.info(f"Ending portfolio value: ${end_value:,.2f}")
    logging.info(f"Total return: {total_return:.2f}%")
    logging.info(f"")
    logging.info(f"Number of transactions: {len(perf[perf['transactions'].apply(len) > 0])}")
    logging.info("="*60)

## Run Backtest

Run the strategy from 2020 to 2023.

In [ ]:
# Run the backtest
results = run_algorithm(
    start=pd.Timestamp('2020-01-01'),
    end=pd.Timestamp('2023-12-31'),
    initialize=initialize,
    handle_data=handle_data,
    analyze=analyze,
    capital_base=100000,  # Start with $100,000
    data_frequency='daily',
    bundle='sharadar',
)

## Analyze Results

Plot portfolio value over time.

In [ ]:
# Plot portfolio value
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Extract date range for title
start_year = results.index[0].year
end_year = results.index[-1].year

# Portfolio value
axes[0].plot(results.index, results['portfolio_value'], linewidth=2)
axes[0].set_ylabel('Portfolio Value ($)', fontsize=12)
axes[0].set_title(f'Buy and Hold: {SYMBOL} ({start_year}-{end_year})', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Format y-axis as currency
axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# Asset price
axes[1].plot(results.index, results['price'], linewidth=2, color='orange')
axes[1].set_ylabel(f'{SYMBOL} Price ($)', fontsize=12)
axes[1].set_xlabel('Date', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics
print("\n" + "="*60)
print(f"{SYMBOL} - SUMMARY STATISTICS")
print("="*60)
print(f"Total Return: {((results['portfolio_value'].iloc[-1] / results['portfolio_value'].iloc[0]) - 1) * 100:.2f}%")
print(f"Max Portfolio Value: ${results['portfolio_value'].max():,.2f}")
print(f"Min Portfolio Value: ${results['portfolio_value'].min():,.2f}")
print(f"Final Portfolio Value: ${results['portfolio_value'].iloc[-1]:,.2f}")
print("="*60)

## View Transactions

See when trades were executed.

In [ ]:
# Extract transactions
transactions = []
for date, row in results.iterrows():
    if len(row['transactions']) > 0:
        for txn in row['transactions']:
            transactions.append({
                'Date': date,
                'Symbol': txn['sid'].symbol,
                'Amount': txn['amount'],
                'Price': txn['price'],
                'Value': txn['amount'] * txn['price']
            })

if transactions:
    txn_df = pd.DataFrame(transactions)
    print("\nTransactions:")
    print(txn_df.to_string(index=False))
else:
    print("\nNo transactions executed.")

## Pyfolio Analysis

Detailed performance analysis using pyfolio library.

In [ ]:
# Import pyfolio and suppress matplotlib info warnings
import warnings
warnings.filterwarnings('ignore')

# Suppress matplotlib.category INFO messages (cosmetic warnings from pyfolio)
import logging
logging.getLogger('matplotlib.category').setLevel(logging.WARNING)

try:
    import pyfolio as pf
    print("✓ Pyfolio imported successfully")
except ImportError:
    print("⚠️  Pyfolio not installed. Install with: pip install pyfolio-reloaded")
    raise

In [ ]:
# Extract returns and positions for pyfolio
returns = results['returns']

# Extract positions (pyfolio needs DOLLAR VALUES, not share counts)
positions_data = []
for date, row in results.iterrows():
    if row['positions']:
        pos_dict = {'cash': row['positions'][0]['amount']}  # cash position
        for pos in row['positions'][1:]:  # skip cash
            if hasattr(pos['sid'], 'symbol'):
                # CRITICAL: Multiply shares by price to get dollar value
                # pyfolio expects position values in dollars, not share counts
                pos_dict[pos['sid'].symbol] = pos['amount'] * pos['last_sale_price']
        positions_data.append((date, pos_dict))

if positions_data:
    positions = pd.DataFrame([p[1] for p in positions_data],
                            index=[p[0] for p in positions_data])
else:
    positions = None

# Extract transactions
transactions_list = []
for date, row in results.iterrows():
    if row['transactions']:
        for txn in row['transactions']:
            transactions_list.append({
                'symbol': txn['sid'].symbol if hasattr(txn['sid'], 'symbol') else str(txn['sid']),
                'amount': txn['amount'],
                'price': txn['price'],
                'value': txn['amount'] * txn['price'],
            })

if transactions_list:
    transactions_pf = pd.DataFrame(transactions_list,
                               index=[date for date, row in results.iterrows()
                                     if row['transactions'] for _ in row['transactions']])
else:
    transactions_pf = None

print(f"✓ Data prepared for pyfolio")
print(f"  Returns: {len(returns)} days")
print(f"  Positions: {len(positions) if positions is not None else 0} days")
print(f"  Transactions: {len(transactions_pf) if transactions_pf is not None else 0} trades")

In [ ]:
# Generate comprehensive tearsheet
print(f"\n{'='*60}")
print(f"PYFOLIO TEARSHEET: {SYMBOL} Buy and Hold Strategy")
print(f"{'='*60}\n")

pf.create_full_tear_sheet(
    returns,
    positions=positions,
    transactions=transactions_pf,
    live_start_date=None,
    round_trips=False,
    estimate_intraday=False,  # Disable intraday detection
)

In [ ]:
# Calculate and display key performance metrics
annual_return = pf.timeseries.annual_return(returns)
sharpe_ratio = pf.timeseries.sharpe_ratio(returns)
max_drawdown = pf.timeseries.max_drawdown(returns)
sortino_ratio = pf.timeseries.sortino_ratio(returns)
calmar_ratio = pf.timeseries.calmar_ratio(returns)
volatility = pf.timeseries.annual_volatility(returns)

print("\n" + "="*60)
print(f"{SYMBOL} - KEY PERFORMANCE METRICS")
print("="*60)
print(f"Annual Return: {annual_return*100:.2f}%")
print(f"Annual Volatility: {volatility*100:.2f}%")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Sortino Ratio: {sortino_ratio:.2f}")
print(f"Calmar Ratio: {calmar_ratio:.2f}")
print(f"Max Drawdown: {max_drawdown*100:.2f}%")
print("="*60)

# Calculate cumulative returns
cum_returns = pf.timeseries.cum_returns(returns)
total_return = cum_returns.iloc[-1]
print(f"\nTotal Return: {total_return*100:.2f}%")
print(f"Final Portfolio Value: ${results['portfolio_value'].iloc[-1]:,.2f}")

## Summary

This notebook demonstrates a complete buy-and-hold backtesting workflow:
- **Configurable**: Change `SYMBOL` variable to test any stock or ETF
- **Comprehensive Analysis**: Portfolio value, price charts, and transactions
- **Pyfolio Integration**: Full tearsheet with risk metrics, drawdowns, and returns analysis
- **Key Metrics**: Sharpe ratio, Sortino ratio, max drawdown, and more

## Next Steps

**To try a different symbol:**
1. Go to the "Configuration" section
2. Change `SYMBOL = 'SPY'` to your desired symbol (e.g., `'AAPL'`, `'NVDA'`)
3. Re-run all cells

**Experiment with:**
- Different time periods (modify `start` and `end` in the backtest)
- Different starting capital (modify `capital_base`)
- Compare multiple symbols by running the notebook multiple times

**Explore more strategies:**
- `03_moving_average_crossover.ipynb` - Active trading strategy
- `04_pyfolio_analysis.ipynb` - Advanced pyfolio usage
- `05_backtesting_101.ipynb` - Strategy development fundamentals